# Character-Level LSTM
На этом семинаре поговорим про рекуррентные нейронные сети (Recurrent Neural Networ, RNN). Мы обучим модель на тексте книги "Анна Каренина", после чего попробуем генерировать новый текст.

**Модель сможет генерировать новый текст на основе текста "Анны Карениной"!**

Можно посмотреть полезную [статью про RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) и [реализацию в Torch](https://github.com/karpathy/char-rnn). 

Ообщая архитектура RNN:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="500">

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Загрузим данные

Загрузим текстовый файл "Анны Карениной".

In [2]:
with open('../input/charrnn/data/data/anna.txt', 'r') as f:
    text = f.read()

Посмотрим первые 100 символов:

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Токенизация

В ячейках ниже создадим два **словаря** для преобразования символов в целые числа и обратно. Кодирование символов как целых чисел упрощает их использование в качестве входных данных в сети.

In [4]:
# кодирование символов в инт и обратно
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

Посмотрим как символы закодировались целыми числами

In [5]:
encoded[:100]

array([50, 38, 10, 18, 56, 21, 61, 82, 59, 57, 57, 57, 29, 10, 18, 18,  6,
       82, 37, 10,  4,  3, 60,  3, 21, 77, 82, 10, 61, 21, 82, 10, 60, 60,
       82, 10, 60,  3, 55, 21, 24, 82, 21, 32, 21, 61,  6, 82, 66, 17, 38,
       10, 18, 18,  6, 82, 37, 10,  4,  3, 60,  6, 82,  3, 77, 82, 66, 17,
       38, 10, 18, 18,  6, 82,  3, 17, 82,  3, 56, 77, 82, 27, 40, 17, 57,
       40, 10,  6, 35, 57, 57, 16, 32, 21, 61,  6, 56, 38,  3, 17])

## Предобработка данных

Как можно видеть на изображении char-RNN выше, сеть ожидает **one-hot encoded** входа, что означает, что каждый символ преобразуется в целое число (через созданный словарь), а затем преобразуется в вектор-столбец, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте создадим для этого функцию.

In [6]:
def one_hot_encode(arr, n_labels):
    # создаем пустой массив
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)    
    # делаем ohe
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    # reshape в исходную форму 
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
test_seq = np.array([[3, 5, 1,9]])
one_hot = one_hot_encode(test_seq, 10)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]]


## Создаем mini-batch'и


Создатдим мини-батчи для обучения. На простом примере они будут выглядеть так:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>
<br>

Возьмем закодированные символы (переданные как параметр `arr`) и разделим их на несколько последовательностей, заданных параметром `batch_size`. Каждая из наших последовательностей будет иметь длину `seq_length`.

### Создани батчей

**1. Первое, что нам нужно сделать, это отбросить часть текста, чтобы у нас были только полные мини-батчи**

Каждый батч содержит $ N\times M $ символов, где $ N $ - это размер батча (количество последовательностей в батче), а $ M $ - длина `seq_length` или количество шагов в последовательности. Затем, чтобы получить общее количество батчей $ K $, которое мы можем сделать из массива `arr`, нужно нацело разделить длину `arr` на количество символов в батче ($N * M$). Когда мы узнаем количество батчей, можно получить общее количество символов, которые нужно сохранить, из `arr`: $ N * M * K $.

**2. После этого нам нужно разделить `arr` на $N$ батчей** 

Это можно сделать с помощью `arr.reshape(size)`, где `size` - это кортеж, содержащий размеры измененного массива. Мы знаем, что нам нужно $ N $ последовательностей в батче, поэтому сделаем его размером первого измерения. Для второго измерения можем использовать «-1» в качестве заполнителя, он заполнит массив соответствующими данными. После этого должен остаться массив $N\times(M * K)$.

**3. Теперь, когда у нас есть этот массив, мы можем перебирать его, чтобы получить наши мини-батчи**

Идея состоит в том, что каждая партия представляет собой окно $ N\times M $ в массиве $ N\times (M * K) $. Для каждого последующего батча окно перемещается на `seq_length`. Мы также хотим создать как входной, так и выходной массивы. Это окно можно сделать с помощью `range`, чтобы делать шаги размером `n_steps` от $ 0 $ до `arr.shape [1]`, общее количество токенов в каждой последовательности. Таким образом, целые числа, которые получены из диапазона, всегда указывают на начало батча, и каждое окно имеет ширину `seq_length`.

> **TODO:** Допишите функцию для создания батчей: 

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Создаем генератор, возвращающий батчи
       размером (batch_size x seq_length) из arr
       
       ---------
       arr: массив, для которого будем делать батчи
       batch_size: размер батча - кол-во последовательностей в батче
       seq_length: кол-во символов в каждой послед батча
    '''
    chars_in_batch = batch_size * seq_length # кол-во символов в батче (во всех послед)
    n_batches = len(arr) // chars_in_batch   # общее кол-во батчей
    arr = arr[:n_batches * chars_in_batch]   # берем ту часть arr, чтобы были целые батчи
    arr = arr.reshape((batch_size, -1))      # разделяем кол-во последовательностей = batch_size 
    
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]           # закодированные символы (фичи)
        y = np.zeros_like(x)                 # символы со сдвигом на 1 (таргеты)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Протестируем

Теперь создадим несколько наборов данных, и проверим, что происходит, когда мы создаем батчи.

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[50 38 10 18 56 21 61 82 59 57]
 [77 27 17 82 56 38 10 56 82 10]
 [21 17 45 82 27 61 82 10 82 37]
 [77 82 56 38 21 82 14 38  3 21]
 [82 77 10 40 82 38 21 61 82 56]
 [14 66 77 77  3 27 17 82 10 17]
 [82  9 17 17 10 82 38 10 45 82]
 [44  1 60 27 17 77 55  6 35 82]]

y
 [[38 10 18 56 21 61 82 59 57 57]
 [27 17 82 56 38 10 56 82 10 56]
 [17 45 82 27 61 82 10 82 37 27]
 [82 56 38 21 82 14 38  3 21 37]
 [77 10 40 82 38 21 61 82 56 21]
 [66 77 77  3 27 17 82 10 17 45]
 [ 9 17 17 10 82 38 10 45 82 77]
 [ 1 60 27 17 77 55  6 35 82 74]]


Если вы правильно реализовали get_batches, результат должен выглядеть примерно так:
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 хотя точные цифры могут отличаться. Убедитесь, что данные сдвинуты на один шаг для `y`!!!

---
## Зададим архитектуру


<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=500px>

### Структура модели

В `__init__` предлагаемая структура выглядит следующим образом:
* Создаваём и храним необходимые словари (уже релизовано)
* Определяем слой LSTM, который принимает в качестве параметров: размер ввода (количество символов), размер скрытого слоя `n_hidden`, количество слоев` n_layers`, вероятность drop-out'а `drop_prob` и логическое значение batch_first (True)
* Определяем слой drop-out с помощью drop_prob
* Определяем полносвязанный слой с параметрами: размер ввода `n_hidden` и размер выхода - количество символов
* Наконец, инициализируем веса

Обратите внимание, что некоторые параметры были названы и указаны в функции `__init__`, их нужно сохранить и использовать, выполняя что-то вроде` self.drop_prob = drop_prob`.

---
### Входы-выходы LSTM

Вы можете создать [LSTM layer](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) следующим образом

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

где `input_size` - это количество символов, которые эта ячейка ожидает видеть в качестве последовательного ввода, а `n_hidden` - это количество элементов в скрытых слоях ячейки. Можно добавить drop-out, добавив параметр `dropout` с заданной вероятностью. Наконец, в функции `forward` мы можем складывать ячейки LSTM в слои, используя `.view`.

Также требуется создать начальное скрытое состояние всех нулей:

```python
self.init_hidden()
```

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # создание словарей для символов
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # определяем LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # определяем дропаут слой
        self.dropout = nn.Dropout(drop_prob)
        
        # опрделяем финальный FC слой
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Прямой проход нейросети'''
                
        # получаем output и новый промежуточный слой
        r_output, hidden = self.lstm(x, hidden)
        
        # пропускаем выходы через дропаут
        out = self.dropout(r_output)
        
        # делаем решейп - получаем число строк = числу объектов в батче, в каждой строке - выходы по конкретному символу
        out = out.contiguous().view(-1, self.n_hidden)
        
        # выходы скрытых состояний пропускаем через фк-слой
        out = self.fc(out)
        
        # вовзвращаем итоговые логиты и финальное скрытое состояние
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Инициализация скрытого состояния '''
        # создание двух тензоров, размером: n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

## Пример работы нашей модели с батчами 
Для понимания структуры модели и процесса обучения сети

In [13]:
# берем батч с кол-вом послед-й = 3 и длиной = 6
x, y = next(get_batches(encoded, 3, 6))
x, y

(array([[50, 38, 10, 18, 56, 21],
        [66,  1, 51, 21, 14, 56],
        [60, 21, 82, 27, 32, 21]]),
 array([[38, 10, 18, 56, 21, 61],
        [ 1, 51, 21, 14, 56, 35],
        [21, 82, 27, 32, 21, 61]]))

In [14]:
# делаем ohe кодирование каждого символа
ohe_x = torch.from_numpy(one_hot_encode(x, len(chars)))
ohe_x, ohe_x.shape

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 torch.Size([3, 6, 83]))

In [15]:
# рандомно инициализируем начальное скрытое состояние для LSTM-слоя
# скрытое сост описывается в данной реализации двумя параметрами h,c
h0 = torch.randn(2, 3, 10)
c0 = torch.randn(2, 3, 10)

# создаем "голый" LSTM-слой с размером входа = числу уник символов, размер скрытого слоя = 10, кол-во LTSM-слоев = 2 
just_lstm = nn.LSTM(len(chars), 10, 2, batch_first=True)

# получаем выходы крайнего слоя сети(в нашем случае-второго) и скрытые состояния для каждого элемента каждой послед-ти в каждом батче(h,c)
output, (hn, cn) = just_lstm(ohe_x,(h0, c0))
output, output.shape

(tensor([[[ 0.0343,  0.0047, -0.3562, -0.0677, -0.2162,  0.0644, -0.0560,
           -0.1192,  0.2907,  0.4693],
          [ 0.0432,  0.0379, -0.0285, -0.0083, -0.1016, -0.0305, -0.0924,
           -0.2044,  0.1201,  0.2923],
          [ 0.0087,  0.0746,  0.0887,  0.0246,  0.0055, -0.0856, -0.1197,
           -0.1254,  0.0343,  0.1540],
          [-0.0202,  0.0978,  0.1564,  0.0359,  0.0554, -0.0872, -0.1346,
           -0.0587, -0.0166,  0.0855],
          [-0.0340,  0.1181,  0.2068,  0.0520,  0.0730, -0.0846, -0.1353,
           -0.0087, -0.0439,  0.0781],
          [-0.0434,  0.1345,  0.2414,  0.0688,  0.0774, -0.0722, -0.1293,
            0.0297, -0.0418,  0.0921]],
 
         [[-0.0091, -0.0742,  0.2105, -0.3617,  0.0749,  0.1683, -0.3839,
           -0.0665,  0.0220,  0.1851],
          [-0.0189, -0.0059,  0.2139, -0.1150,  0.1075, -0.0083, -0.1848,
            0.0128,  0.0209,  0.1614],
          [-0.0252,  0.0555,  0.2466, -0.0017,  0.0964, -0.0559, -0.1822,
            0.0360,

In [16]:
# containing the final hidden state for each element in the sequenc
hn, hn.shape

(tensor([[[-0.0731, -0.0884, -0.1309, -0.1100, -0.2039, -0.0496, -0.0475,
            0.0463, -0.1645,  0.1421],
          [-0.1208, -0.0712, -0.2103, -0.0777, -0.1898, -0.0607, -0.0109,
           -0.0080, -0.1912,  0.1071],
          [-0.0627, -0.1520, -0.2189, -0.1021, -0.2187, -0.0550, -0.0602,
            0.0023, -0.1318,  0.1787]],
 
         [[-0.0434,  0.1345,  0.2414,  0.0688,  0.0774, -0.0722, -0.1293,
            0.0297, -0.0418,  0.0921],
          [-0.0409,  0.1412,  0.2729,  0.0890,  0.0762, -0.0717, -0.1383,
            0.0621, -0.0221,  0.1063],
          [-0.0951,  0.1607,  0.2851,  0.0721,  0.1141, -0.0682, -0.0889,
            0.0705, -0.0818,  0.0931]]], grad_fn=<StackBackward0>),
 torch.Size([2, 3, 10]))

In [17]:
# containing the final cell state for each element in the sequence
cn, cn.shape

(tensor([[[-0.1519, -0.1802, -0.2485, -0.2007, -0.4782, -0.1164, -0.0759,
            0.0839, -0.3283,  0.2355],
          [-0.2064, -0.1238, -0.4940, -0.1292, -0.3691, -0.1296, -0.0164,
           -0.0136, -0.4459,  0.2223],
          [-0.1278, -0.3141, -0.4277, -0.1842, -0.5101, -0.1262, -0.0969,
            0.0041, -0.2626,  0.2978]],
 
         [[-0.1110,  0.2744,  0.4516,  0.1397,  0.1516, -0.1289, -0.2816,
            0.0805, -0.0744,  0.1881],
          [-0.1053,  0.2948,  0.5190,  0.1841,  0.1525, -0.1308, -0.3028,
            0.1693, -0.0393,  0.2137],
          [-0.2433,  0.3269,  0.5481,  0.1504,  0.2305, -0.1223, -0.1883,
            0.1929, -0.1429,  0.1900]]], grad_fn=<StackBackward0>),
 torch.Size([2, 3, 10]))

In [18]:
# делаем решейп - получаем число строк = числу объектов в батче, в каждой строке - выходы по конкретному символу
output = output.contiguous().view(-1, 10)
output, output.shape

(tensor([[ 0.0343,  0.0047, -0.3562, -0.0677, -0.2162,  0.0644, -0.0560, -0.1192,
           0.2907,  0.4693],
         [ 0.0432,  0.0379, -0.0285, -0.0083, -0.1016, -0.0305, -0.0924, -0.2044,
           0.1201,  0.2923],
         [ 0.0087,  0.0746,  0.0887,  0.0246,  0.0055, -0.0856, -0.1197, -0.1254,
           0.0343,  0.1540],
         [-0.0202,  0.0978,  0.1564,  0.0359,  0.0554, -0.0872, -0.1346, -0.0587,
          -0.0166,  0.0855],
         [-0.0340,  0.1181,  0.2068,  0.0520,  0.0730, -0.0846, -0.1353, -0.0087,
          -0.0439,  0.0781],
         [-0.0434,  0.1345,  0.2414,  0.0688,  0.0774, -0.0722, -0.1293,  0.0297,
          -0.0418,  0.0921],
         [-0.0091, -0.0742,  0.2105, -0.3617,  0.0749,  0.1683, -0.3839, -0.0665,
           0.0220,  0.1851],
         [-0.0189, -0.0059,  0.2139, -0.1150,  0.1075, -0.0083, -0.1848,  0.0128,
           0.0209,  0.1614],
         [-0.0252,  0.0555,  0.2466, -0.0017,  0.0964, -0.0559, -0.1822,  0.0360,
           0.0073,  0.1066],
 

In [22]:
# применяем ФК-слой к выходам по каждому элементу 
# и получаем логиты вероятностей след элементов
just_fc = nn.Linear(10, len(chars))
res = just_fc(output)
res, res.shape

(tensor([[ 0.0835,  0.1542,  0.0207,  ..., -0.3010, -0.3587, -0.1807],
         [ 0.1259,  0.1088,  0.1051,  ..., -0.2967, -0.3181, -0.2657],
         [ 0.1095,  0.0956,  0.1968,  ..., -0.3081, -0.2664, -0.3092],
         ...,
         [ 0.0431,  0.0641,  0.3614,  ..., -0.3191, -0.2067, -0.4160],
         [ 0.0370,  0.0698,  0.3448,  ..., -0.3261, -0.2007, -0.4000],
         [ 0.0372,  0.0670,  0.3266,  ..., -0.3379, -0.2049, -0.3922]],
        grad_fn=<AddmmBackward0>),
 torch.Size([18, 83]))

In [23]:
# ответы для каждого из 3(batch size) * 6(seq_len) = 18 элементов 
torch.from_numpy(y).view(3*6)

tensor([38, 10, 18, 56, 21, 61,  1, 51, 21, 14, 56, 35, 21, 82, 27, 32, 21, 61])

In [24]:
# считаем лосс по батчу, далее бэк-проп
nn.CrossEntropyLoss()(res, torch.from_numpy(y).view(3*6))

tensor(4.4358, grad_fn=<NllLossBackward0>)

### То же самое внутри нашего франкенштейна
Внутри модели идет и слой лстм, и фк-слой и решейпинг

In [26]:
net = CharRNN(chars, n_hidden=10, n_layers=2, drop_prob=0)

In [27]:
output, (hn, cn) = net(ohe_x,(h0, c0))

In [28]:
output, output.shape

(tensor([[-0.1803, -0.2769,  0.4006,  ..., -0.1604, -0.2651, -0.3944],
         [-0.2628, -0.2175,  0.4230,  ..., -0.0868, -0.2442, -0.3540],
         [-0.2785, -0.2026,  0.4353,  ..., -0.0254, -0.2635, -0.3452],
         ...,
         [-0.3254, -0.1689,  0.4939,  ...,  0.0110, -0.2737, -0.3406],
         [-0.3181, -0.1756,  0.4895,  ...,  0.0188, -0.2767, -0.3465],
         [-0.3127, -0.1810,  0.4802,  ...,  0.0180, -0.2770, -0.3476]],
        grad_fn=<AddmmBackward0>),
 torch.Size([18, 83]))

In [29]:
# считаем лосс по батчу, далее бэк-проп
nn.CrossEntropyLoss()(output, torch.from_numpy(y).view(3*6))

tensor(4.4621, grad_fn=<NllLossBackward0>)

## Обучим модель

Во время обучения нужно установить количество эпох, скорость обучения и другие параметры.

Используем оптимизатор Adam и кросс-энтропию, считаем loss и, как обычно, выполняем back propagation.

Пара подробностей об обучении:
> * Во время цикла мы отделяем скрытое состояние от его истории; на этот раз установив его равным новой переменной * tuple *, потому что скрытое состояние LSTM, является кортежем скрытых состояний.
* Мы используем [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) чтобы избавиться от взрывающихся градиентов.

In [30]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Функция тренировки модели
        ---------
        
        net: CharRNN - класс модели
        data: текст для обучения модели text data to train the network
        epochs: кол-во эпох
        batch_size: кол-во последовательностей в мини-батче
        seq_length: кол-во символов в каждой последовательности в мини-батче
        lr: learning rate
        clip: gradient clipping
        val_frac: доля валидационного датасета // Fraction of data to hold out for validation
        print_every: кол-во шагов для вывода логов // Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # разделяем на трени и вал выборки
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars) # уник символы текста
    for e in range(epochs):
        # инициализируем скрытое состояние
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot кодирование и перевод в тензоры
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # обнуляем старые градиенты
            net.zero_grad()
            
            # прямой проход модели по батчу
            output, h = net(inputs, h)
            
            # считаем лосс и делаем бэк-проп
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Определим модель

Теперь мы можем создать модель с заданными гиперпараметрами. Определим размеры мини-батчей.

In [32]:
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


### Установим гиперпараметры

In [33]:
batch_size = 128
seq_length = 100
n_epochs = 20 

train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=20)

Epoch: 1/20... Step: 20... Loss: 3.1508... Val Loss: 3.1298
Epoch: 1/20... Step: 40... Loss: 3.1124... Val Loss: 3.1205
Epoch: 1/20... Step: 60... Loss: 3.1153... Val Loss: 3.1153
Epoch: 1/20... Step: 80... Loss: 3.1172... Val Loss: 3.1065
Epoch: 1/20... Step: 100... Loss: 3.0701... Val Loss: 3.0573
Epoch: 1/20... Step: 120... Loss: 2.9059... Val Loss: 2.8791
Epoch: 2/20... Step: 140... Loss: 2.7197... Val Loss: 2.6579
Epoch: 2/20... Step: 160... Loss: 2.5703... Val Loss: 2.5201
Epoch: 2/20... Step: 180... Loss: 2.4748... Val Loss: 2.4459
Epoch: 2/20... Step: 200... Loss: 2.4105... Val Loss: 2.3865
Epoch: 2/20... Step: 220... Loss: 2.3480... Val Loss: 2.3239
Epoch: 2/20... Step: 240... Loss: 2.3079... Val Loss: 2.2726
Epoch: 2/20... Step: 260... Loss: 2.2221... Val Loss: 2.2230
Epoch: 3/20... Step: 280... Loss: 2.2259... Val Loss: 2.1782
Epoch: 3/20... Step: 300... Loss: 2.1687... Val Loss: 2.1354
Epoch: 3/20... Step: 320... Loss: 2.1208... Val Loss: 2.1007
Epoch: 3/20... Step: 340... 

## Checkpoint

После обучения сохраним модель, чтобы можно было загрузить ее позже. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [34]:
# change the name, for saving multiple files
model_name = 'lstm_20th_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Делаем предсказания

Теперь, когда мы обучили модель, сделаем предсказание следующих символов! Для предсказания мы передаем последний символ, и сеть предсказывает следующий символ, который мы потом передаем снова на вхол и получаем еще один предсказанный символ и так далее...

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые наиболее вероятные символы $K$. Это не позволит сети выдавать нам совершенно абсурдные прогнозы, а также позволит внести некоторый шум и случайность в выбранный текст. Узнать больше [можно здесь](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [36]:
def predict(net, char, h=None, top_k=None):
        ''' На вход подается символ, ф-я предсказания след символа
            На выход - предсказанный символ и новое скрытое состояние
        '''
        
        # переводим новый сивол в ohe-вектор
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # получаем выходы модели
        out, h = net(inputs, h)

        # находим вероятности по логитам
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # выбираем след элемент с некоторой степенью рандома
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # возвращаем предсказ символ и новое скрыт сост
        return net.int2char[char], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы. 

In [63]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # режим предсказывания
    
    # для начало "поглотим" данный нам текст, чтобы логически оттталкиваться от него
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # теперь будем передавать последний символ и получать следующий
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [64]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna
trying to talk about the most the thousand, and as that to be sure
of that morning. And how to see him we consider to be, but he saw to
and her husband his senticious cross to the position to him.

"What do you think that he won't come to me, but I'm going over them with
a long while, and there's not the first to set yourself," said
Stepan Arkadyevitch, walking into the carriage of such things
on the mother--to a little child only were answer to his hand.

The sorn on the carriase was not settled and still higher from his huge
face, he was as he had at all that she said. After all astors, and a son of
starting it had taken the marshal, but ansters of corneds, true that
they were angroused her shoulders, and the mather, with a smothed
soulded hands. And all the complete people would be an harishing all the promate and sore
of theirse or this conversation to him.

"Well, I'm all at our forever that you are thought in which I should hear that your husband
send him all the council to 

## Loading a checkpoint

In [42]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('lstm_20th_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [44]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said:

These walked about. He had a good table wingon.

"That's in a minute," said Sergey Ivanovitch, with a smile, "than
the world of his wife has not telling the most, bus silly and
an implasting anything, at all, and simply so sorry as I'm ashamed."

"You're anyone to do?" added, "the sorts of a man, thonger of it
to some tears of her. But that's as if the side of it.

"Well, I'd been such as all so to do all, you're all all over. We are
never coming. What do you true that I have a last times it was," she
said to himself at the same time in the most dismiling, "I have seen a long while,
and he say, that I do not anyway, and wouldn't you any soul."

"Yes, yes; I'm asked that," said Vronsky.

The card at once so in the sorts and would not be calmed at the
stretters which cared to the steps with the plate, and the
singer the farely was as he had stooped him. And that she had not
came out of his strange, some old man had been satisfaction
that we have been to be already she sa

### Полезные ссылки:


*   [Блог-пост Christopher'а Olah'а по LSTM](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)